In [344]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import make_column_transformer


from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt


In [142]:
ls ../data

country_code_lookup.csv  yemen_batch0.csv  yemen_batch0.pkl  yemen_batch1.pkl
event_code_lookup.csv    yemen_batch0.pk   yemen_batch1.csv


In [221]:
yemen_df0 = pd.read_csv('../data/yemen_batch0.csv', index_col=0)
yemen_df0.head(2)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.0


# Generator for Batch Processing Test

In [316]:
def load_data(filename, idx):
    df = pd.read_csv('../data/{}{}.csv'.format(filename, idx), index_col=0)
    features = ['QuadClass','GoldsteinScale','NumArticles','AvgTone']
    X_train = df[features]
    y_train = df['mean']
    return X_train, y_train

In [300]:
def batch_generator(filename,n):
    idx = 0
    while idx<n:
        yield load_data(filename=filename, idx=idx)
        idx+=1

In [306]:
filename = 'yemen_batch'
gen = batch_generator(filename, 2)

In [229]:
# next(gen)

# One Hot Encoding for Categorical Variables

In [232]:
features_ohe = yemen_df0.Actor1Name.value_counts()[yemen_df0.Actor1Name.value_counts().values > 50].keys()

In [241]:
yemen_df0_sub = yemen_df0[yemen_df0.Actor1Name.isin(features_ohe)]
yemen_df0_sub.head()

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.0
2014-02-19,LIBYA,YEMENI,1,2.8,2,-6.102747,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,1,-6.243603,False,0,9,0.0
2014-02-19,LIBYA,YEMENI,1,2.8,1,-6.243603,False,0,9,0.0


In [245]:
yemen_df0_sub.fillna('None', inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [264]:
col = ['Actor1Name', 'Actor2Name', 'IsHumanitarianCrisis']
column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown = 'ignore'),  col),
                        remainder='passthrough')
column_trans.fit_transform(yemen_df0_sub)

<12274x498 sparse matrix of type '<class 'numpy.float64'>'
	with 119264 stored elements in Compressed Sparse Row format>

In [254]:
onehotencodeer = OneHotEncoder(handle_unknown='ignore')
onehotencodeer.fit_transform(yemen_df0_sub)

<12274x5255 sparse matrix of type '<class 'numpy.float64'>'
	with 122740 stored elements in Compressed Sparse Row format>

In [262]:
len(yemen_df0['mean'])

18435

In [236]:
# yemen_df0.Actor2Name.value_counts()[yemen_df0.Actor2Name.value_counts().values > 50].keys()

# NN model with Batch Processing

In [320]:
model = Sequential()
model.add(Dense(units=1, input_dim=4))
model.compile(loss='mean_squared_error', optimizer='sgd')

In [321]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [360]:
features = ['QuadClass','GoldsteinScale','NumArticles','AvgTone']
X_train = yemen_df0[features]
y_train = yemen_df0['mean']

In [324]:
X_train.head(2)

,QuadClass,GoldsteinScale,NumArticles,AvgTone
SQLDATE,,,,
2014-02-19,3,-6.5,3,-2.103250
2014-02-19,1,1.9,2,-6.102747


In [291]:
col = ['Actor1Name', 'Actor2Name']
X_train.fillna('None', inplace=True)
column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown = 'ignore'),  col))
X_train_ohe = column_trans.fit_transform(X_train)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [292]:
model.fit(X_train_ohe, y_train)

18435/18435 [==============================] - 1s 45us/sample - loss: 0.0057


In [322]:
model.fit_generator(batch_generator(filename, 2), steps_per_epoch=2)

2/2 [==============================] - 0s 131ms/step - loss: 51.5327


In [313]:
len(column_trans.get_feature_names())

1177

In [325]:
model.predict(X_train)

array([[ 2.16504  ],
       [-1.0053207],
       [-1.713297 ],
       ...,
       [-9.454945 ],
       [ 2.8743365],
       [-7.5633974]], dtype=float32)

# Time-series Forecasting

In [337]:
yemen_df0.head(2)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.0


In [362]:
X_train.tail(5)

,QuadClass,GoldsteinScale,NumArticles,AvgTone
SQLDATE,,,,
2015-02-28,4,-5.0,4,0.397812
2015-02-28,1,7.0,4,-2.431611
2015-02-28,1,3.5,10,2.054795
2015-02-28,4,-10.0,10,-8.490566
2015-02-28,1,7.0,2,0.826446


In [363]:
label_col = ['QuadClass']
label_encoder = LabelEncoder()

X_train[label_col] = label_encoder.fit_transform(X_train[label_col])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [364]:
X_train.tail(5)

,QuadClass,GoldsteinScale,NumArticles,AvgTone
SQLDATE,,,,
2015-02-28,3,-5.0,4,0.397812
2015-02-28,0,7.0,4,-2.431611
2015-02-28,0,3.5,10,2.054795
2015-02-28,3,-10.0,10,-8.490566
2015-02-28,0,7.0,2,0.826446


In [366]:
stdscaler = StandardScaler()
X_train_scaled = stdscaler.fit_transform(X_train)
X_train_scaled

array([[ 0.715327  , -1.09962278, -0.57030264,  0.38771791],
       [-0.87462396,  0.4354165 , -0.74302367, -0.90855672],
       [-0.87462396,  0.59988499, -0.74302367, -0.90855672],
       ...,
       [-0.87462396,  0.72780493,  0.63874458,  1.7353797 ],
       [ 1.51030248, -1.73922247,  0.63874458, -1.68247161],
       [-0.87462396,  1.36740463, -0.74302367,  1.3372604 ]])

# Appendix

In [73]:
def sample_generator():
    i = 0
    while True:
        i += 1
        yield(i)
        

In [82]:
gen = sample_generator()

In [84]:
next(gen)

2

In [ ]:
def load_data(train_df,idx,
              batch_size):
    df = pd.read_csv(
                  Train_df, skiprows=idx*batch_size,
                  nrows=batch_size)
    x = df.iloc[:,1:]
         
    y = df.iloc[:,0]
    return (np.array(x), np_utils.to_categorical(y))

In [91]:
def load_data(filename, idx):
    df = pd.read_csv('../data/{}{}.csv'.format(filename, idx), index_col=0)
    return df.head(1)

idx = '0'
filename = 'yemen_batch'
load_data(filename, idx)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.10325,False,0,9,0.0


In [92]:
def sample_generator(filename):
    idx = 0
    while True:
        yield(load_data(filename, idx))
    

In [93]:
gen = sample_generator(filename)

In [94]:
gen

<generator object sample_generator at 0x7faf0e267bf8>

In [95]:
for x in range(5):
    print(x)
    

0
1
2
3
4


In [135]:
def sam_generator(limit):
    i = 0
    while i<limit:
        i += 1
        yield i




In [136]:
gen = sam_generator(4)

In [141]:
next(gen)

StopIteration: 

In [315]:
def load_data_old(filename, idx):
    df = pd.read_csv('../data/{}{}.csv'.format(filename, idx), index_col=0)
    features = ['Actor1Name','Actor2Name','QuadClass','GoldsteinScale','NumArticles','AvgTone']
    categorical_features = ['Actor1Name','Actor2Name']
    column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown = 'ignore'),  categorical_features),
                        remainder='passthrough')
    df.fillna('None', inplace=True)
    X_train = df[features]
    X_train_ohe = column_trans.fit_transform(X_train).toarray()
    y_train = df['mean']
    return X_train_ohe, y_train